In [4]:
import selenium.webdriver as wb  # 爬虫框架/自动化测试框架
import time  # 随机休眠，防止反爬机制
import numpy as np  # 处理数组用
import csv  # 处理数据，存储和读取CSV文件
import re  # 正则表达式


def init_csv(filepath='test.csv'):
    headers = ['UP昵称', '会员等级', '会员类别', '关注', '粉丝', '获赞', '播放', '生日', '投稿']#最后文件的标题行
    with open(filepath, 'w', newline='', encoding='utf-8')as f:#w指直接写入
        f_csv = csv.writer(f)
        f_csv.writerow(headers)#创建文件，写入标题行


def write_csv(filepath, datas):
    with open(filepath, 'a+', newline='', encoding='utf-8')as f:#
        f_csv = csv.writer(f)
        f_csv.writerow(datas)#每爬取一行数据就写入


def read_uid(filepath):
    x = []
    with open(filepath, 'r', newline='', encoding='utf-8')as f:
        csv_read = csv.reader(f)
        for line in csv_read:
            line[0] = re.findall('\d+', line[0])[0]#正则化处理
            x.append(line[0])
    return x


class Spider(object):#定义一个爬虫类

    def __init__(self, display=True) -> None:
        super().__init__()
        option = wb.ChromeOptions()
        # 忽略driver自身的警告|错误等输出
        option.add_experimental_option(
            'excludeSwitches', ['enable-logging'])
        # 禁用图片加载,加速爬取url等页面数据,图片数据待爬取url后由requests库实现
        # option.add_experimental_option(
        #     'prefs', {"profile.managed_default_content_settings.images": 2})
        if display == False:
            option.headless = True
            print('程序运行，设置为无窗口模式')

        ################    Chrome程序地址改这里
        option.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        # 浏览器驱动器初始化，需指明路径与参数
        ################    Chrome驱动器地址改这里，改try里的那个就行
        try:
            self.driver = wb.Chrome(
                r"D:\Chromedriver\chromedriver.exe", options=option)
        except:
            self.driver = wb.Chrome(
                r"C:\Users\Ludy_Lab\Desktop\workspace\assets\chromedriver.exe", options=option)

    def login(self):
        self.driver.get('https://space.bilibili.com/' + '51896064')
        self.random_sleep(2)
        login_elm = self.driver.find_element_by_css_selector(
            '#internationalHeader > div > div > div.nav-user-center > div.user-con.logout > div:nth-child(1) > div > span > div > span')
        #找到登陆按钮的元素
        login_elm.click()#要求点击
        wds = self.driver.window_handles#点击之后弹出到一个新的网页
        self.driver.switch_to.window(wds[-1])#返回到上一个网页
        self.random_sleep(2)
        username_elm: wb.remote.webdriver.WebElement = self.driver.find_element_by_css_selector(
            '#login-username')
        password_elm: wb.remote.webdriver.WebElement = self.driver.find_element_by_css_selector(
            '#login-passwd')
        login2_elm: wb.remote.webdriver.WebElement = self.driver.find_element_by_css_selector(
            '#geetest-wrap > div > div.btn-box > a.btn.btn-login')#登录界面的登陆按钮

        ################################
        # 账号密码改这里
        username_elm.send_keys('15029220381')#输入账号
        password_elm.send_keys('g123456')#输入密码
        self.random_sleep(1)
        login2_elm.click()
        # 8秒的时间进行人机结合登录=_=
        time.sleep(12)

    def random_sleep(self, delay=3):
        tmp = np.random.random()#随机数0-1
        time.sleep(delay * (tmp + 0.5))#每爬一个网页随机暂停几秒

    def fetch_by_UID(self, UID='51896064'):
        self.driver.get('https://space.bilibili.com/' + UID)
        self.random_sleep(3)

        up_name_elm = self.driver.find_element_by_css_selector('#h-name')
        print(up_name_elm.text)
        up_name = up_name_elm.text

        up_level_elm: wb.remote.webdriver.WebElement = self.driver.find_element_by_css_selector(
            '#app > div.h > div.wrapper > div.h-inner > div.h-user > div > div.h-basic > div:nth-child(1) > a.h-level.m-level')
        print(up_level_elm.get_attribute('lvl'))
        up_level = up_level_elm.get_attribute('lvl')

        up_member_elm = self.driver.find_element_by_css_selector(
            '#app > div.h > div.wrapper > div.h-inner > div.h-user > div > div.h-basic > div:nth-child(1) > a.h-vipType')
        print(up_member_elm.text)
        up_member = up_member_elm.text

        up_follow_elm = self.driver.find_element_by_css_selector('#n-gz')
        print(up_follow_elm.text)
        up_follow = up_follow_elm.text

        up_followby_elm = self.driver.find_element_by_css_selector('#n-fs')
        print(up_followby_elm.text)
        up_followby = up_followby_elm.text

        up_getzan_elm = self.driver.find_element_by_css_selector('#n-bf')
        print(up_getzan_elm.text)
        up_getzan = up_getzan_elm.text

        up_numofplay_elm = self.driver.find_element_by_xpath(
            '/html/body/div[2]/div[2]/div/div[1]/div[3]/div[2]/p[2]')
        print(up_numofplay_elm.text)
        up_numofplay = up_numofplay_elm.text

        birthday = 'None'
        try:
            birthday = self.driver.find_element_by_css_selector(
                '#page-index > div.col-2 > div.section.user > div.info > div.content > div > div > div.item.birthday > span.text').text
        except:
            pass
        print(birthday)

        up_tougao_elm = self.driver.find_element_by_css_selector(
            '#navigator > div > div.n-inner.clearfix > div.n-tab-links > a.n-btn.n-video.n-audio.n-article.n-album > span.n-num').text
        tougao = up_tougao_elm
        xiangbu = 'Error'
        try:
            up_tougao_elm = self.driver.find_element_by_xpath(
                '//*[@id="navigator"]/div/div[1]/div[1]/a[3]/span[2]')
            up_tougao_elm.click()
            self.random_sleep(3)
            up_xiangbu_elm = self.driver.find_element_by_css_selector(
                '#page-video > div > div.contribution-sidenav > div > div > ul > li:nth-child(4) > span')
            xiangbu = up_xiangbu_elm.text
        except:
            pass
        if xiangbu != 'Error':
            tougao = str(int(tougao) - int(xiangbu))
        print(tougao)
        self.random_sleep(2)
        return (up_name, up_level, up_member, up_follow, up_followby, up_getzan, up_numofplay, birthday, tougao)


if __name__ == "__main__":#从此开始执行
    # 用来保存爬虫结果的文件是下面这个，可以不新建
    csv_path = r'.\bilibli_ups2.csv'
    # 用来读取UID列表的csv文件是这个，excel要先另存为成csv，并且不要有标题行
    uid_path = r'.\UID100.csv'
    uid_list = read_uid(uid_path)#返回成一个列表
    worker = Spider(display=True)#spider是前面定义的一个爬虫类
    try:
        worker.login()
    except:
        print('登录失败')
        time.sleep(10)
        exit()

    try:
        init_csv(csv_path)#执行初始化CSV的函数
    except:
        pass

    for i in range(len(uid_list)):
        try:
            write_csv(csv_path, worker.fetch_by_UID(uid_list[i]))
            print('*' * 20 + '\n', i, 'success')
        except:
            print(i, 'error')
    time.sleep(10)
    # worker.random_sleep()

FileNotFoundError: [Errno 2] No such file or directory: '.\\UID100.csv'